<a href="https://colab.research.google.com/github/kangseungwan5446/LUT/blob/main/LUT2006(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 kB 2.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sdf
import numpy as np
import pandas as pd

# Load 2006 LUT for interpolation based on iteration

# Pressure range [MPa] from 2006 LUT, convert to [Pa]
P = np.array((0.10,0.30,0.50,1.0,2.0,3.0,5.0,7.0,10.0,12.0,14.0,16.0,18.0,20.0,21.0))*1e6

# Mass Flux range [kg/m^2-s] from 2006 LUT
G = np.array((0,50,100,300,500,750,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,7500,8000))

# Quality range from 2006 LUT
x = np.array((-0.50,-0.40,-0.30,-0.20,-0.15,-0.10,-0.05,0.00,0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,0.60,0.70,0.80,0.90,1.00))

# Critical heat flux [kW/m^2] from 2006 LUT, convert to [W/m^2]
q_raw=np.loadtxt('/content/drive/MyDrive/2006LUTdata.txt')*1e3

# Convert the imported array into a (MxNxQ) where:
#   M is number of mass flux divisions
#   N is number of quality divisions
#   Q is number of pressure divisions
lenG = len(G)
lenx = len(x)
lenP = len(P)
q = np.zeros((lenG,lenx,lenP))
for i in range(lenG):
    for j in range(lenx):
        for k in range(lenP):
            q[i,j,k] = q_raw[i + k*lenG,j]

# Create the datasets:
ds_G = sdf.Dataset('G', data=G, unit='kg/(m2.s)', is_scale=True, display_name='Mass Flux')
ds_x = sdf.Dataset('x', data=x, unit='1', is_scale=True, display_name='Quality')
ds_P = sdf.Dataset('P', data=P, unit='Pa', is_scale=True, display_name='Pressure')
ds_q = sdf.Dataset('q', data=q, unit='W/m2', scales=[ds_G,ds_x,ds_P])

# Create the root group and write the file:
g2 = sdf.Group('/', comment='2006 CHF LUT', datasets=[ds_G,ds_x,ds_P,ds_q])
sdf.save('2006LUT.sdf', g2)

print(g2)

<SDF Group "/": [G, x, P, q]>


In [ ]:
import numpy as np
import sdf
import pandas as pd

sdf_file = '2006LUT.sdf'
loaded_data = sdf.load(sdf_file)

file_path = '/content/drive/MyDrive/data through LUT.xlsx'
df = pd.read_excel(file_path)
df = df.values
print(df)
P_data = df[:,2]
P_data = P_data.tolist()
x_data = df[:,1]
x_data = x_data.tolist()
G_data = df[:,0]
G_data = G_data.tolist()
print(P_data)
print(x_data)
print(G_data)

ds_G = loaded_data['G']
ds_x = loaded_data['x']
ds_P = loaded_data['P']
ds_q = loaded_data['q']

print(ds_G)
print(ds_x)
print(ds_P)
print(ds_q)

print(ds_G.data.shape)
print(ds_x.data.shape)
print(ds_P.data.shape)
print(ds_q.data.shape)

# # print('함수 돌아가기전!!')



def search(target_G, target_x, target_P):
    target_massflux = target_G
    target_qulity = target_x
    target_pressure = target_P

    list_index_G = []
    list_index_x = []
    list_index_P = []


    # 가장가까운 인덱스 찾기
    index_target_G = np.argmin(np.abs(ds_G.data-target_massflux))
    index_target_x = np.argmin(np.abs(ds_x.data-target_qulity))
    index_target_P = np.argmin(np.abs(ds_P.data-target_pressure))
    # print(f'G 인덱스 ',index_target_G)
    # print(index_target_G)
    # print(index_target_x)
    # print(index_target_P)

    index_target_G2 = index_target_G
    index_target_x2 = index_target_x
    index_target_P2 = index_target_P

    list_index_G.append(index_target_G)
    list_index_x.append(index_target_x)
    list_index_P.append(index_target_P)

    # 그 다음으로 가까운 인덱스 찾기
    if target_massflux - ds_G.data[index_target_G] > 0:
        index_target_G2 = index_target_G + 1
    elif target_massflux - ds_G.data[index_target_G] < 0:
        index_target_G2 = index_target_G  - 1
    else:
        pass


    if target_qulity - ds_x.data[index_target_x] >0:
        index_target_x2 = index_target_x + 1
    elif target_qulity - ds_x.data[index_target_x] <0:
        index_target_x2 = index_target_x - 1
    else:
        pass

    if target_pressure - ds_P.data[index_target_P] > 0:
        index_target_P2 = index_target_P + 1
    elif target_pressure - ds_P.data[index_target_P] < 0:
        index_target_P2 = index_target_P - 1
    else:
        pass

    list_index_G.append(index_target_G2)
    list_index_x.append(index_target_x2)
    list_index_P.append(index_target_P2)

    def remove_duplicates_sort(seq):
        seen = set()
        # 중복을 제거하면서 순서를 유지합니다.
        no_duplicates = [x for x in seq if not (x in seen or seen.add(x))]
        # 중복이 제거된 리스트를 정렬합니다.
        sorted_list = sorted(no_duplicates)
        return sorted_list

    # 예시 리스트에 대해 함수를 호출합니다.
    list_index_G = remove_duplicates_sort(list_index_G)
    list_index_x = remove_duplicates_sort(list_index_x)
    list_index_P = remove_duplicates_sort(list_index_P)
    # print(list_index_G)
    # print(list_index_x)
    # print(list_index_P)

    q_value = []

    for i in list_index_G:
        for j in list_index_x:
            for k in list_index_P:
                q_value.append(ds_q.data[i,j,k])
    q_value = np.array(q_value)
    q_value = q_value.reshape(len(list_index_G),len(list_index_x), len(list_index_P))




    if len(list_index_G) > 1:

        delta_G = ds_G.data[list_index_G[1]]-ds_G.data[list_index_G[0]]
        delta_q_G = q_value[1,:,:]-q_value[0,:,:]
        target_delta_G = target_massflux - ds_G.data[list_index_G[0]]
        q_G = q_value[0,:,:] + (delta_q_G/delta_G) * target_delta_G
        q_value = q_G
        q_value.reshape(len(list_index_x),len(list_index_P))

    else:
        q_value = q_value.reshape(len(list_index_x),len(list_index_P))
        # print(q_value.shape)

    if len(list_index_x) > 1:

        delta_x = ds_x.data[list_index_x[1]]-ds_x.data[list_index_x[0]]
        delta_q_x = q_value[1,:]-q_value[0,:]
        target_delta_x = target_qulity - ds_x.data[list_index_x[0]]
        q_x = q_value[0,:] + (delta_q_x/delta_x) * target_delta_x
        q_value = q_x

    else:
        q_value = q_value.reshape(len(list_index_P))
        # print(f'q모양',{q_value.shape})

    if len(list_index_P) > 1:

        delta_P = ds_P.data[list_index_P[1]]-ds_P.data[list_index_P[0]]
        delta_q_P = q_value[1]-q_value[0]
        target_delta_P = target_pressure - ds_P.data[list_index_P[0]]
        q_P = q_value[0] + (delta_q_P/delta_P) * target_delta_P
        q_value = q_P
    else:
        pass

    return q_value


#     print(q_value)




# print(f'응애',G_data[0])




def search_list(x,y,z):

    result = []
    for i in range(len(P_data)):

        value = search(x[i],y[i],z[i])
        result.append(value)
    return result

data = search_list(G_data,x_data,P_data)
df = pd.DataFrame(data)
file_path = '/content/hi.xlsx'
df.to_excel(file_path,index= False)




[[ 5.600e+03 -1.041e-01  3.900e+05]
 [ 6.700e+03 -5.960e-02  3.100e+05]
 [ 4.300e+03 -3.950e-02  3.300e+05]
 ...
 [ 2.000e+03 -1.090e-02  1.010e+06]
 [ 2.000e+03 -2.180e-02  1.010e+06]
 [ 2.000e+03 -4.340e-02  1.010e+06]]
[390000.0, 310000.0, 330000.0, 620000.0, 640000.0, 640000.0, 910000.0, 10000000.0, 10000000.0, 10000000.0, 12000000.0, 12000000.0, 12000000.0, 12000000.0, 14000000.0, 14000000.0, 14000000.0, 16000000.0, 16000000.0, 16000000.0, 18000000.0, 18000000.0, 18000000.0, 20000000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 100000.0, 2140000.0, 2140000.0, 2140000.0, 3450000.0, 3450000.0, 3960000.0, 4030000.0000000005, 3560000.0, 3830000.0, 3900000.0, 3760000.0, 3720000.0, 3760000.0, 3830000.0, 3930000.0, 3960000.0, 3830000.0, 3860000.0, 3860000.0, 3860000.0, 3860000.0, 3900000.0, 3450000.0, 3450000.0, 3450000.0, 3450000.0, 3450000.